In [1]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf

tf.enable_eager_execution()

import pandas as pd
import numpy as np
import random as random
import sonnet as snt


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [13]:
frame = tf.cast(obs[0]['X'], tf.float32)

In [14]:
delta = tf.cast(obs[0]['B'], tf.float32)

In [144]:
future_price = tf.cast(obs[0]['A'], tf.float32)

Instructions for updating:
Colocations handled automatically by placer.


In [47]:
conv_out = tf.nn.relu(conv_out)
conv_out = snt.BatchFlatten()(conv_out)

In [48]:
conv_out = snt.Linear(256)(conv_out)

In [51]:
action = tf.layers.dense(
    conv_out, 
    1, 
    activation=tf.nn.tanh, 
    kernel_initializer = tf.contrib.layers.variance_scaling_initializer(),
    kernel_regularizer = tf.contrib.layers.l2_regularizer(0.1)
)

In [ ]:
Batch = collections.namedtuple(
    'ActorOutput', 
    [
        'size',
        'frame',
        'last_w',
        'next_price',
        'delta',
        'com',
        'decay'
    ]
)

In [280]:
def train(batch):
    
        action, price = agent(
            batch.frame, 
            batch.last_w
        )
        
        # distribution of portfolio 
        # in next step
        future_omega = 0
        
        w_t = future_omega[:batch.size-1]
        w_t1 = action[1:batch.size]
        mu = 1 - (tf.reduce_sum(
            tf.abs(w_t1[:, 1:]-w_t[:, 1:]),
            axis=1
        )*batch.com + batch.decay)
        
        # determine deltas from prices
        
        free_rets = tf.reduce_sum(
            tf.concat(
                [
                    action*batch.delta,
                    tf.ones([batch.size,1])
                ], 
                axis=1), 
            axis=1
        )
        
        rets = free_rets * ( 
            tf.concat(
                [
                tf.ones(1),
                mu
                ],
                axis=0
            )
        )
        
        # Mean returns
        mean = tf.reduce_mean(rets)
        log_mean = tf.reduce_mean(tf.log(rets))
        log_mean_free = tf.reduce_mean(free_rets)
        
        # Standard Deviation of returns
        stdev = tf.sqrt(tf.reduce_mean(
           (rets - mean) ** 2
        ))
        
        # Sharpe ratio
        sharp_ratio = (mean - 1) / stdev
        
        # Portfolio value
        p_value = tf.reduce_prod(rets)
        
        loss = -tf.reduce_mean(tf.log(rets))
        
        tf.train.AdamOptimizer(learning_rate).minimize(
            loss, 
            global_step=self.__global_step
        )
        
        return {
            ''
        }

In [ ]:
def make_test_batch():
    batch = []
    for f in range(10):
        start_date = df_mini.iloc[f]['time']

        a = df_mini.iloc[f:f+window_size,:]
        b = df_norm[df_norm['time']>=start_date][:window_size]

        del a['time']

        c = df_mini.iloc[f+window_size+1][["close_price"]].values # next price
        d = c/df_mini.iloc[f+window_size][["close_price"]].values # fraction delta

        batch.append({
            'a': a,
            'b': b,
            'c': c,
            'd': d
        })

In [ ]:
for x in range(10):
    batch = make_batch(batch_size=100):
    results = train(batch)    

In [ ]:
# test
batch = make_batch()
for i in batch:
    pass